In [40]:
import pandas as pd 
import numpy as np 

In [41]:
# Load the data from the CSV file
ufc_stat = pd.read_csv('ufc_stat.csv')

In [42]:
# Load the dataset
data = pd.read_csv('ufc_stat.csv')

# Convert columns to numeric and replace non-numeric values with NaN
cols_to_convert = [
    'Title Fight', 'Main Event', 'F1 Lay Pray', 'F2 Lay Pray', 'Winner LP',
    'F1 Failed LP', 'F2 Failed LP', 'Decisions', 'Stinker', 'F1 Frozen', 'F2 Frozen',
    'Potential Robbery', 'Stand Up', 'F1 Active', 'F2 Active', 'Comeback Win', 'BangFest',
    'F1 Head Hunter', 'F2 Head Hunter', 'F1 Leg Kicker', 'F2 Leg Kicker', 'Paul Craig',
    'Massacre', 'F1 Beatdown', 'F2 Beatdown', 'Time Elapsed'
]

for col in cols_to_convert:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Define a list of weight classes in the dataset
weight_classes = data['Weight Class'].unique()


# Create a new DataFrame to store averages
averages_columns = [
    'Weight Class', 'Average Title Fight', 'Average Main Event', 'Average F1 Lay Pray',
    'Average F2 Lay Pray', 'Average Winner LP', 'Average F1 Failed LP', 'Average F2 Failed LP',
    'Average Decision', 'Average Stinker', 'Average F1 Frozen', 'Average F2 Frozen',
    'Average Potential Robbery', 'Average Stand Up', 'Average F1 Active', 'Average F2 Active',
    'Average Comeback Win', 'Average BangFest', 'Average F1 Head Hunter', 'Average F2 Head Hunter',
    'Average F1 Leg Kicker', 'Average F2 Leg Kicker', 'Average Paul Craig', 'Average Massacre',
    'Average F1 Beatdown', 'Average F2 Beatdown', 'Average Time Elapsed'
]
averages_data = pd.DataFrame(columns=averages_columns)
# Calculate averages for each weight class
for wc in weight_classes:
    wc_data = data[data['Weight Class'] == wc]
    
    # Calculate averages
    averages_row = {
        'Weight Class': wc,
        'Average Title Fight': wc_data['Title Fight'].mean(),
        'Average Main Event': wc_data['Main Event'].mean(),
        'Average F1 Lay Pray': wc_data['F1 Lay Pray'].mean(),
        'Average F2 Lay Pray': wc_data['F2 Lay Pray'].mean(),
        'Average Winner LP': wc_data['Winner LP'].mean(),
        'Average F1 Failed LP': wc_data['F1 Failed LP'].mean(),
        'Average F2 Failed LP': wc_data['F2 Failed LP'].mean(),
        'Average Decision': wc_data['Decisions'].mean(),
        'Average Stinker': wc_data['Stinker'].mean(),
        'Average F1 Frozen': wc_data['F1 Frozen'].mean(),
        'Average F2 Frozen': wc_data['F2 Frozen'].mean(),
        'Average Potential Robbery': wc_data['Potential Robbery'].mean(),
        'Average Stand Up': wc_data['Stand Up'].mean(),
        'Average F1 Active': wc_data['F1 Active'].mean(),
        'Average F2 Active': wc_data['F2 Active'].mean(),
        'Average Comeback Win': wc_data['Comeback Win'].mean(),
        'Average BangFest': wc_data['BangFest'].mean(),
        'Average F1 Head Hunter': wc_data['F1 Head Hunter'].mean(),
        'Average F2 Head Hunter': wc_data['F2 Head Hunter'].mean(),
        'Average F1 Leg Kicker': wc_data['F1 Leg Kicker'].mean(),
        'Average F2 Leg Kicker': wc_data['F2 Leg Kicker'].mean(),
        'Average Paul Craig': wc_data['Paul Craig'].mean(),
        'Average Massacre': wc_data['Massacre'].mean(),
        'Average F1 Beatdown': wc_data['F1 Beatdown'].mean(),
        'Average F2 Beatdown': wc_data['F2 Beatdown'].mean(),
        'Average Time Elapsed': wc_data['Time Elapsed'].mean()
    }
    
    # Append the calculated averages to the new DataFrame
    averages_data = averages_data.append(averages_row, ignore_index=True)

# Save the new DataFrame to a CSV file
averages_data.to_csv('averages.csv', index=False)


C:\Users\khoob\AppData\Local\Temp\ipykernel_28072\1171274885.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  averages_data = averages_data.append(averages_row, ignore_index=True)
C:\Users\khoob\AppData\Local\Temp\ipykernel_28072\1171274885.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  averages_data = averages_data.append(averages_row, ignore_index=True)
C:\Users\khoob\AppData\Local\Temp\ipykernel_28072\1171274885.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  averages_data = averages_data.append(averages_row, ignore_index=True)
C:\Users\khoob\AppData\Local\Temp\ipykernel_28072\1171274885.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pan

In [43]:
# Load the 'ufc_stat.csv' dataset
ufc_data = pd.read_csv('ufc_stat.csv')

# Load the 'averages.csv' dataset
averages_data = pd.read_csv('averages.csv')

# List of weight classes
weight_classes = ["FYW", "BW", "FW", "LW", "WW", "MW", "LHW", "HW"]

# Function to calculate the average strikes thrown per fighter for each weight class
def average_strikes_thrown_per_fighter(wc):
    wc_data = ufc_data[ufc_data['Weight Class'] == wc]
    avg_strikes = (wc_data['F1 Total Strikes'].mean() + wc_data['F2 Total Strikes'].mean()) / 2
    return avg_strikes

# Calculate the average strikes thrown per fighter for each weight class and add to the DataFrame
averages_data['Average Strikes Thrown per Fighter'] = averages_data['Weight Class'].apply(average_strikes_thrown_per_fighter)

# Save the updated DataFrame to a new CSV file
averages_data.to_csv('averages.csv', index=False)


In [44]:
# Function to calculate the average sig strikes landed per fighter for each weight class
def average_sig_strikes_per_fighter(wc):
    wc_data = ufc_data[ufc_data['Weight Class'] == wc]
    avg_sig_strikes = (wc_data['F1 Sig Strikes'].mean() + wc_data['F2 Sig Strikes'].mean()) / 2
    return avg_sig_strikes

# Calculate the average sig strikes landed per fighter for each weight class and add to the DataFrame
averages_data['Average Sig Strikes Landed per Fighter'] = averages_data['Weight Class'].apply(average_sig_strikes_per_fighter)

# Save the updated DataFrame to a new CSV file
averages_data.to_csv('averages.csv', index=False)

In [45]:
# Function to calculate the average TDA per fighter for each weight class
def average_tda_per_fighter(wc):
    wc_data = ufc_data[ufc_data['Weight Class'] == wc]
    avg_tda = (wc_data['F1 TDA'].mean() + wc_data['F2 TDA'].mean()) / 2
    return avg_tda

# Calculate the average TDA per fighter for each weight class and add to the DataFrame
averages_data['Average TDA per Fighter'] = averages_data['Weight Class'].apply(average_tda_per_fighter)

# Save the updated DataFrame to a new CSV file
averages_data.to_csv('averages.csv', index=False)

In [46]:
# Function to calculate the average TDL per fighter for each weight class
def average_tdl_per_fighter(wc):
    wc_data = ufc_data[ufc_data['Weight Class'] == wc]
    avg_tdl = (wc_data['F1 TDL'].mean() + wc_data['F2 TDL'].mean()) / 2
    return avg_tdl

# Calculate the average TDA per fighter for each weight class and add to the DataFrame
averages_data['Average TDL per Fighter'] = averages_data['Weight Class'].apply(average_tdl_per_fighter)

# Save the updated DataFrame to a new CSV file
averages_data.to_csv('averages.csv', index=False)

In [47]:
# Function to calculate the average Control Time per fighter for each weight class
def average_cntrl_per_fighter(wc):
    wc_data = ufc_data[ufc_data['Weight Class'] == wc]
    avg_cntrl = (wc_data['F1 Control Time'].mean() + wc_data['F2 Control Time'].mean()) / 2
    return avg_cntrl

# Calculate the average Control Time per fighter for each weight class and add to the DataFrame
averages_data['Average Control Time per Fighter'] = averages_data['Weight Class'].apply(average_cntrl_per_fighter)

# Save the updated DataFrame to a new CSV file
averages_data.to_csv('averages.csv', index=False)

In [48]:
# Function to calculate the average sig strikes landed per second
def average_sig_strike_second_per_fighter(wc):
    wc_data = ufc_data[ufc_data['Weight Class'] == wc]
    avg_sig_strike_second = (wc_data['F1 Sig Strike/Sec'].mean() + wc_data['F2 Sig Strike/Sec'].mean()) / 2
    return avg_sig_strike_second

# Calculate the average sig Strikes landed per second per fighter for each weight class and add to the DataFrame
averages_data['Average Sig Strikes Landed per Second'] = averages_data['Weight Class'].apply(average_sig_strike_second_per_fighter)

# Save the updated DataFrame to a new CSV file
averages_data.to_csv('averages.csv', index=False)

In [49]:
# Function to calculate the average strikes thrown per second
def average_strike_second_per_fighter(wc):
    wc_data = ufc_data[ufc_data['Weight Class'] == wc]
    avg_strike_second = (wc_data['F1 Strike/Sec'].mean() + wc_data['F2 Strike/Sec'].mean()) / 2
    return avg_strike_second

# Calculate the average  Strikes thrown per second per fighter for each weight class and add to the DataFrame
averages_data['Average Strikes Thrown per Second'] = averages_data['Weight Class'].apply(average_strike_second_per_fighter)

# Save the updated DataFrame to a new CSV file
averages_data.to_csv('averages.csv', index=False)

In [50]:
# Function to calculate the average Decisions
def average_decisions(wc):
    wc_data = ufc_data[ufc_data['Weight Class'] == wc]
    avg_decisions = (wc_data['Decisions'].mean())
    return avg_decisions

# Calculate the average  Decisios for each weight class and add to the DataFrame
averages_data['Proportion Decisions'] = averages_data['Weight Class'].apply(average_decisions)

# Save the updated DataFrame to a new CSV file
averages_data.to_csv('averages.csv', index=False)

In [51]:
# Function to calculate the average Submissions
def average_submission(wc):
    wc_data = ufc_data[ufc_data['Weight Class'] == wc]
    avg_submission = (wc_data['Submission'].mean())
    return avg_submission

# Calculate the average  Submissions for each weight class and add to the DataFrame
averages_data['Proportion Submissions'] = averages_data['Weight Class'].apply(average_submission)

# Save the updated DataFrame to a new CSV file
averages_data.to_csv('averages.csv', index=False)

In [52]:
# Function to calculate the average KOs
def average_ko(wc):
    wc_data = ufc_data[ufc_data['Weight Class'] == wc]
    avg_ko = (wc_data['KO'].mean())
    return avg_ko

# Calculate the average  KO/TKO for each weight class and add to the DataFrame
averages_data['Proportion KO/TKO'] = averages_data['Weight Class'].apply(average_ko)

# Save the updated DataFrame to a new CSV file
averages_data.to_csv('averages.csv', index=False)

In [53]:
# Get unique fighter names from both columns
fighters_1 = ufc_stat["Fighter 1"].unique()
fighters_2 = ufc_stat["Fighter 2"].unique()

# Combine the two arrays of unique fighter names and remove duplicates
fighters = pd.Series(pd.concat([pd.Series(fighters_1), pd.Series(fighters_2)], ignore_index=True).unique(), name="Fighter Name")

# Save the unique fighter names to a CSV file
fighters.to_csv("fighters.csv", index=False)



In [54]:
fighters = pd.read_csv('fighters.csv')

In [55]:
# Load the fighters.csv file
fighters = pd.read_csv("fighters.csv")

# Load the ufc_stat.csv file
ufc_stat = pd.read_csv("ufc_stat.csv")

# Loop through each fighter and populate the weight class column
for index, fighter in fighters.iterrows():
    name = fighter["Fighter Name"]
    weight_class = ufc_stat.loc[(ufc_stat["Fighter 1"] == name) | (ufc_stat["Fighter 2"] == name), "Weight Class"].values[0]
    fighters.loc[fighters["Fighter Name"] == name, "Weight Class"] = weight_class

# Save the updated fighters DataFrame to a CSV file
fighters.to_csv("fighters.csv", index=False)

In [56]:
# Load the unique fighter names from the CSV file
fighters = pd.read_csv("fighters.csv")

# Re-index the fighters DataFrame
fighters = fighters.reset_index(drop=True)

# Loop through each fighter and populate the number of fights column
for index, fighter in fighters.iterrows():
    name = fighter["Fighter Name"]
    num_fights = len(ufc_stat[(ufc_stat["Fighter 1"] == name) | (ufc_stat["Fighter 2"] == name)])
    fighters.loc[fighters["Fighter Name"] == name, "# of Fights"] = num_fights

# Save the updated fighters DataFrame to a CSV file
fighters.to_csv("fighters.csv", index=False)


In [57]:
# Create a new column to hold the number of wins for each fighter
fighters["Wins"] = 0

# Loop through each fighter and populate the wins column
for index, fighter in fighters.iterrows():
    name = fighter["Fighter Name"]
    wins = len(ufc_stat.loc[((ufc_stat["Fighter 1"] == name) & (ufc_stat["Winner"] == "F1")) | ((ufc_stat["Fighter 2"] == name) & (ufc_stat["Winner"] == "F2"))])
    fighters.loc[fighters["Fighter Name"] == name, "Wins"] = wins

# Save the updated fighters DataFrame to a CSV file
fighters.to_csv("fighters.csv", index=False)

In [58]:
# Function to calculate decision wins for a fighter
def calculate_decision_wins(fighter_name, ufc_stat):
    decision_wins = 0

    # Check if the fighter is Fighter 1 or Fighter 2 in each row
    f1_mask = ufc_stat['Fighter 1'] == fighter_name
    f2_mask = ufc_stat['Fighter 2'] == fighter_name

    # Filter the rows where the fighter is either Fighter 1 or Fighter 2
    fighter_rows = ufc_stat[f1_mask | f2_mask]

    for index, row in fighter_rows.iterrows():
        if row['Decisions'] == 1:  # If the fight went to decision
            if row['Winner'] == 'F1' and f1_mask[index]:  # If the fighter was Fighter 1 and won
                decision_wins += 1
            elif row['Winner'] == 'F2' and f2_mask[index]:  # If the fighter was Fighter 2 and won
                decision_wins += 1

    return decision_wins

# Add a new column to the fighters dataset with the decision wins
fighters['Decision Wins'] = fighters['Fighter Name'].apply(lambda x: calculate_decision_wins(x, ufc_stat))

# Save the updated fighters dataset to a new CSV file
fighters.to_csv('fighters.csv', index=False)


In [59]:
# Function to calculate KO/TKO wins for a fighter
def calculate_KO_wins(fighter_name, ufc_stat):
    KO_wins = 0

    # Check if the fighter is Fighter 1 or Fighter 2 in each row
    f1_mask = ufc_stat['Fighter 1'] == fighter_name
    f2_mask = ufc_stat['Fighter 2'] == fighter_name

    # Filter the rows where the fighter is either Fighter 1 or Fighter 2
    fighter_rows = ufc_stat[f1_mask | f2_mask]

    for index, row in fighter_rows.iterrows():
        if row['KO'] == 1:  # If the fight finished by KO/TKO
            if row['Winner'] == 'F1' and f1_mask[index]:  # If the fighter was Fighter 1 and won
                KO_wins += 1
            elif row['Winner'] == 'F2' and f2_mask[index]:  # If the fighter was Fighter 2 and won
                KO_wins += 1

    return KO_wins

# Add a new column to the fighters dataset with the KO wins
fighters['KO/TKO Wins'] = fighters['Fighter Name'].apply(lambda x: calculate_KO_wins(x, ufc_stat))

# Save the updated fighters dataset to a new CSV file
fighters.to_csv('fighters.csv', index=False)

In [60]:
# Function to calculate submission wins for a fighter
def calculate_sub_wins(fighter_name, ufc_stat):
    sub_wins = 0

    # Check if the fighter is Fighter 1 or Fighter 2 in each row
    f1_mask = ufc_stat['Fighter 1'] == fighter_name
    f2_mask = ufc_stat['Fighter 2'] == fighter_name

    # Filter the rows where the fighter is either Fighter 1 or Fighter 2
    fighter_rows = ufc_stat[f1_mask | f2_mask]

    for index, row in fighter_rows.iterrows():
        if row['Submission'] == 1:  # If the fight finished by KO/TKO
            if row['Winner'] == 'F1' and f1_mask[index]:  # If the fighter was Fighter 1 and won
                sub_wins += 1
            elif row['Winner'] == 'F2' and f2_mask[index]:  # If the fighter was Fighter 2 and won
                sub_wins += 1

    return sub_wins

# Add a new column to the fighters dataset with the Submission wins
fighters['Submission Wins'] = fighters['Fighter Name'].apply(lambda x: calculate_sub_wins(x, ufc_stat))

# Save the updated fighters dataset to a new CSV file
fighters.to_csv('fighters.csv', index=False)

In [61]:
# Create a new column for average fight duration

# Loop through each fighter and calculate their average fight duration
for index, fighter in fighters.iterrows():
    name = fighter["Fighter Name"]
    avg_fight_duration = ufc_stat.loc[(ufc_stat["Fighter 1"] == name) | (ufc_stat["Fighter 2"] == name), "Time Elapsed"].mean()
    fighters.loc[fighters["Fighter Name"] == name, "avg fight duration"] = avg_fight_duration

# Save the updated fighters DataFrame to a CSV file
fighters.to_csv("fighters.csv", index=False)

In [62]:
# Define a function to calculate average strikes per fight
def avg_strikes_per_fight(fighter_name):
    # Get the total number of strikes thrown by the fighter as Fighter 1
    f1_strikes = ufc_stat.loc[(ufc_stat["Fighter 1"] == fighter_name), "F1 Total Strikes"].sum()
    
    # Get the total number of strikes thrown by the fighter as Fighter 2
    f2_strikes = ufc_stat.loc[(ufc_stat["Fighter 2"] == fighter_name), "F2 Total Strikes"].sum()
    
    # Get the total number of fights the fighter was involved in
    num_fights = len(ufc_stat.loc[(ufc_stat["Fighter 1"] == fighter_name) | (ufc_stat["Fighter 2"] == fighter_name)])
    
    # Calculate the average number of strikes per fight
    avg_strikes = (f1_strikes + f2_strikes) / num_fights
    
    return avg_strikes

# Add a new column "Avg Strikes Per Fight"
fighters["Avg Strikes Per Fight"] = fighters["Fighter Name"].apply(avg_strikes_per_fight)

# Save the updated fighters DataFrame to a CSV file
fighters.to_csv("fighters.csv", index=False)

In [63]:
# Define a function to calculate average sig strikes per fight
def avg_sig_strikes_per_fight(fighter_name):
    # Get the total number of sig strikes thrown by the fighter as Fighter 1
    f1_sig_strikes = ufc_stat.loc[(ufc_stat["Fighter 1"] == fighter_name), "F1 Sig Strikes"].sum()
    
    # Get the total number of sig strikes thrown by the fighter as Fighter 2
    f2_sig_strikes = ufc_stat.loc[(ufc_stat["Fighter 2"] == fighter_name), "F2 Sig Strikes"].sum()
    
    # Get the total number of fights the fighter was involved in
    num_fights = len(ufc_stat.loc[(ufc_stat["Fighter 1"] == fighter_name) | (ufc_stat["Fighter 2"] == fighter_name)])
    
    # Calculate the average number of sig strikes per fight
    avg_sig_strikes = (f1_sig_strikes + f2_sig_strikes) / num_fights
    
    return avg_sig_strikes

# Add a new column "Avg Strikes Per Fight"
fighters["Avg Sig Strikes Per Fight"] = fighters["Fighter Name"].apply(avg_sig_strikes_per_fight)

# Save the updated fighters DataFrame to a CSV file
fighters.to_csv("fighters.csv", index=False)

In [64]:
# Define a function to calculate average takedown attempts per fight
def avg_takedown_attempt_per_fight(fighter_name):
    # Get the total number of takedown attempts by the fighter as Fighter 1
    f1_tda = ufc_stat.loc[(ufc_stat["Fighter 1"] == fighter_name), "F1 TDA"].sum()
    
    # Get the total number of takedown attempts by the fighter as Fighter 2
    f2_tda = ufc_stat.loc[(ufc_stat["Fighter 2"] == fighter_name), "F2 TDA"].sum()
    
    # Get the total number of fights the fighter was involved in
    num_fights = len(ufc_stat.loc[(ufc_stat["Fighter 1"] == fighter_name) | (ufc_stat["Fighter 2"] == fighter_name)])
    
    # Calculate the average number of takedown attempts per fight
    avg_tda = (f1_tda + f2_tda) / num_fights
    
    return avg_tda

# Add a new column "Avg Takedown Attempts Per Fight"
fighters["Avg TDA Per Fight"] = fighters["Fighter Name"].apply(avg_takedown_attempt_per_fight)

# Save the updated fighters DataFrame to a CSV file
fighters.to_csv("fighters.csv", index=False)

In [65]:
# Define a function to calculate average takedown landed per fight
def avg_takedown_landed_per_fight(fighter_name):
    # Get the total number of takedowns landed by the fighter as Fighter 1
    f1_tdl = ufc_stat.loc[(ufc_stat["Fighter 1"] == fighter_name), "F1 TDL"].sum()
    
    # Get the total number of takedowns landed by the fighter as Fighter 2
    f2_tdl = ufc_stat.loc[(ufc_stat["Fighter 2"] == fighter_name), "F2 TDL"].sum()
    
    # Get the total number of fights the fighter was involved in
    num_fights = len(ufc_stat.loc[(ufc_stat["Fighter 1"] == fighter_name) | (ufc_stat["Fighter 2"] == fighter_name)])
    
    # Calculate the average number of takedowns landed per fight
    avg_tdl = (f1_tdl + f2_tdl) / num_fights
    
    return avg_tdl

# Add a new column "Avg takedowns landed Per Fight"
fighters["Avg TDL Per Fight"] = fighters["Fighter Name"].apply(avg_takedown_landed_per_fight)

# Save the updated fighters DataFrame to a CSV file
fighters.to_csv("fighters.csv", index=False)

In [66]:
# Create a new column "average control time"
fighters["average control time"] = 0

# Loop through each fighter and populate the "average control time" column
for index, fighter in fighters.iterrows():
    name = fighter["Fighter Name"]
    control_time = ufc_stat.loc[(ufc_stat["Fighter 1"] == name) & (ufc_stat["F1 Control Time"] > 0), "F1 Control Time"].sum() + ufc_stat.loc[(ufc_stat["Fighter 2"] == name) & (ufc_stat["F2 Control Time"] > 0), "F2 Control Time"].sum()
    num_fights = fighter["# of Fights"]
    if num_fights > 0:
        fighters.loc[fighters["Fighter Name"] == name, "average control time"] = control_time / num_fights

# Save the updated dataset to a new csv file called "fighters2.csv"
fighters.to_csv("fighters.csv", index=False)

In [67]:
# Function to count the number of times a fighter has lay and prayed 
def count_LP(fighter_name, ufc_stat):
    LP_count = 0

    # Check if the fighter is Fighter 1 or Fighter 2 in each row
    f1_mask = ufc_stat['Fighter 1'] == fighter_name
    f2_mask = ufc_stat['Fighter 2'] == fighter_name

    # Filter the rows where the fighter is either Fighter 1 or Fighter 2
    fighter_rows = ufc_stat[f1_mask | f2_mask]

    for index, row in fighter_rows.iterrows():
        if row['F1 Lay Pray'] == 1 and f1_mask[index]:  # If the fighter was Fighter 1 and lay and prayed
            LP_count += 1
        elif row['F2 Lay Pray'] == 1 and f2_mask[index]:  # If the fighter was Fighter 2 and lay and prayed
            LP_count += 1

    return LP_count

# Add a new column to the fighters dataset with the LP count
fighters['LP Count'] = fighters['Fighter Name'].apply(lambda x: count_LP(x, ufc_stat))

# Save the updated fighters dataset to a new CSV file
fighters.to_csv('fighters.csv', index=False)


In [68]:
# Create a new column for strikes absorbed per fight
fighters["strikes absorbed per fight"] = 0

# Loop through each fighter and populate the strikes absorbed per fight column
for index, fighter in fighters.iterrows():
    name = fighter["Fighter Name"]
    strikes_absorbed = ufc_stat.loc[(ufc_stat["Fighter 1"] == name), "F2 Sig Strikes"].sum() + ufc_stat.loc[(ufc_stat["Fighter 2"] == name), "F1 Sig Strikes"].sum()
    num_fights = fighter["# of Fights"]
    if num_fights > 0:
        fighters.loc[fighters["Fighter Name"] == name, "strikes absorbed per fight"] = strikes_absorbed / num_fights
# Save the updated fighters dataset to a new CSV file
fighters.to_csv('fighters.csv', index=False)


In [69]:
# Function to count the number of times a fighter was involved in a bangfest
def count_bangfest(fighter_name, ufc_stat):
    bangfest_count = 0

    # Check if the fighter is Fighter 1 or Fighter 2 in each row
    f1_mask = ufc_stat['Fighter 1'] == fighter_name
    f2_mask = ufc_stat['Fighter 2'] == fighter_name

    # Filter the rows where the fighter is either Fighter 1 or Fighter 2
    fighter_rows = ufc_stat[f1_mask | f2_mask]

    for index, row in fighter_rows.iterrows():
        if row['BangFest'] == 1:  # If the fight was a bangfest
            bangfest_count += 1

    return bangfest_count

# Add a new column to the fighters dataset with the bangfest count
fighters['Bangfest Count'] = fighters['Fighter Name'].apply(lambda x: count_bangfest(x, ufc_stat))

# Save the updated fighters dataset to a new CSV file
fighters.to_csv('fighters.csv', index=False)


In [70]:
# Function to count the number of times a fighter was involved in a stinker
def count_stinker(fighter_name, ufc_stat):
    stinker_count = 0

    # Check if the fighter is Fighter 1 or Fighter 2 in each row
    f1_mask = ufc_stat['Fighter 1'] == fighter_name
    f2_mask = ufc_stat['Fighter 2'] == fighter_name

    # Filter the rows where the fighter is either Fighter 1 or Fighter 2
    fighter_rows = ufc_stat[f1_mask | f2_mask]

    for index, row in fighter_rows.iterrows():
        if row['Stinker'] == 1:  # If the fight was a stinker
            stinker_count += 1

    return stinker_count

# Add a new column to the fighters dataset with the stinker count
fighters['Stinker Count'] = fighters['Fighter Name'].apply(lambda x: count_stinker(x, ufc_stat))

# Save the updated fighters dataset to a new CSV file
fighters.to_csv('fighters.csv', index=False)

In [71]:
# Function to count the number of times a fighter was involved in a potential robbery
def count_rob(fighter_name, ufc_stat):
    rob_count = 0

    # Check if the fighter is Fighter 1 or Fighter 2 in each row
    f1_mask = ufc_stat['Fighter 1'] == fighter_name
    f2_mask = ufc_stat['Fighter 2'] == fighter_name

    # Filter the rows where the fighter is either Fighter 1 or Fighter 2
    fighter_rows = ufc_stat[f1_mask | f2_mask]

    for index, row in fighter_rows.iterrows():
        if row['Potential Robbery'] == 1:  # If the fight was a stinker
            rob_count += 1

    return rob_count

# Add a new column to the fighters dataset with the robbery count
fighters['Potential Robbery Count'] = fighters['Fighter Name'].apply(lambda x: count_rob(x, ufc_stat))

# Save the updated fighters dataset to a new CSV file
fighters.to_csv('fighters.csv', index=False)

In [72]:
# Function to count the number of times a fighter was involved in a stand up fight
def count_standup(fighter_name, ufc_stat):
    standup_count = 0

    # Check if the fighter is Fighter 1 or Fighter 2 in each row
    f1_mask = ufc_stat['Fighter 1'] == fighter_name
    f2_mask = ufc_stat['Fighter 2'] == fighter_name

    # Filter the rows where the fighter is either Fighter 1 or Fighter 2
    fighter_rows = ufc_stat[f1_mask | f2_mask]

    for index, row in fighter_rows.iterrows():
        if row['Stand Up'] == 1:  # If the fight was a bangfest
            standup_count += 1

    return standup_count

# Add a new column to the fighters dataset with the standup count
fighters['Standup Count'] = fighters['Fighter Name'].apply(lambda x: count_standup(x, ufc_stat))

# Save the updated fighters dataset to a new CSV file
fighters.to_csv('fighters.csv', index=False)

In [73]:
# Function to count the number of times a fighter was involved in a comeback
def count_comeback(fighter_name, ufc_stat):
    comeback_count = 0

    # Check if the fighter is Fighter 1 or Fighter 2 in each row
    f1_mask = ufc_stat['Fighter 1'] == fighter_name
    f2_mask = ufc_stat['Fighter 2'] == fighter_name

    # Filter the rows where the fighter is either Fighter 1 or Fighter 2
    fighter_rows = ufc_stat[f1_mask | f2_mask]

    for index, row in fighter_rows.iterrows():
        if row['Comeback Win'] == 1:  # If the fight was a bangfest
            comeback_count += 1

    return comeback_count

# Add a new column to the fighters dataset with the Comeback count
fighters['Comeback Count'] = fighters['Fighter Name'].apply(lambda x: count_comeback(x, ufc_stat))

# Save the updated fighters dataset to a new CSV file
fighters.to_csv('fighters.csv', index=False)

In [74]:
# Function to count the number of times a fighter was involved in a massacre
def count_massacre(fighter_name, ufc_stat):
    massacre_count = 0

    # Check if the fighter is Fighter 1 or Fighter 2 in each row
    f1_mask = ufc_stat['Fighter 1'] == fighter_name
    f2_mask = ufc_stat['Fighter 2'] == fighter_name

    # Filter the rows where the fighter is either Fighter 1 or Fighter 2
    fighter_rows = ufc_stat[f1_mask | f2_mask]

    for index, row in fighter_rows.iterrows():
        if row['Massacre'] == 1:  # If the fight was a bangfest
            massacre_count += 1

    return massacre_count

# Add a new column to the fighters dataset with the massacre count
fighters['Massacre Count'] = fighters['Fighter Name'].apply(lambda x: count_massacre(x, ufc_stat))

# Save the updated fighters dataset to a new CSV file
fighters.to_csv('fighters.csv', index=False)

In [75]:
import pandas as pd

# Read the dataset
data = pd.read_csv("ufc_stat.csv")

# Define the initial columns
initial_columns = ["Fighter 1", "Fighter 2", "Winner", "Weight Class", "Venue"]

# Define the columns for the fight_categories.csv file
fight_categories_columns = initial_columns + [
    "Title Fight", "Main Event", "F1 Lay Pray", "F2 Lay Pray", "Winner LP",
    "F1 Failed LP", "F2 Failed LP", "Decisions", "Stinker", "F1 Frozen",
    "F2 Frozen", "Potential Robbery", "Stand Up", "F1 Active", "F2 Active",
    "Comeback Win", "BangFest", "F1 Head Hunter", "F2 Head Hunter",
    "F1 Leg Kicker", "F2 Leg Kicker", "Paul Craig", "Massacre",
    "F1 Beatdown", "F2 Beatdown", "Submission", "KO"
]

# Define the columns for the fight_stats.csv file
fight_stats_columns = initial_columns + [
    "Time Elapsed", "F1 TDA", "F1 TDL", "F1 Control Time", "F2 TDA", "F2 TDL",
    "F2 Control Time", "F1 Total Strikes", "F1 Sig Strikes", "F1 Head Strike",
    "F1 Body Strike", "F1 Leg Strike", "F2 Total Strikes", "F2 Sig Strikes",
    "F2 Head Strike", "F2 Body Strike", "F2 Leg Strike", "F1 KD", "F2 KD",
    "F1 Strike Acc%", "F1 TD Acc%", "F1 HS%", "F1 BS%", "F1 LS%",
    "F1 Strike/Sec", "F1 Sig Strike/Sec", "F2 Strike Acc%", "F2 TD Acc%",
    "F2 HS%", "F2 BS%", "F2 LS%", "F2 Strike/Sec", "F2 Sig Strike/Sec",
    "F1 Strike Diff", "F1 Sig Strike Diff", "F2 Strike Diff", "F2 Sig Strike Diff",
    "Method of Victory ", "Gender"
]

# Create the fight_categories.csv file
fight_categories = data[fight_categories_columns]
fight_categories.to_csv("fight_categories.csv", index=False)

# Create the fight_stats.csv file
fight_stats = data[fight_stats_columns]
fight_stats.to_csv("fight_stats.csv", index=False)
